In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d

import pickle
import os
import torch

import pandas as pd

import plotly.io as pio
pio.renderers.default = 'notebook+vscode'
from sklearn.cluster import KMeans

#import sys
#sys.path.insert(0, '/Users/danielverschueren/Documents/Sonalis/code/registration/registration/')
#sys.path.insert(0, '/Users/danielverschueren/Documents/Sonalis/code/registration/')

from point_cloud_torch import affinePC_torch, apply_affine_torch, apply_affine_inverse_torch
from point_cloud import plot_PCs, affinePC

# the fit function in affinePC calls on minimize_parallel from optimparallel
# make sure that package is installed
def create_cylinder(z, r):
    """
    construct a point cloud of a cylinder with radius r and length len(z)
    """
    phi = np.linspace(0, 2*np.pi, 100)
    xyz = np.zeros((phi.shape[0]*z.shape[0], 3))
    
    for i in range(phi.shape[0]):
        j = i*z.shape[0]
        xyz[j:j+z.shape[0],0] = np.sin(phi[i])*r 
        xyz[j:j+z.shape[0],1] = np.cos(phi[i])*r
        xyz[j:j+z.shape[0],2] = z

    return xyz

def pgy2npy(pgy_file):
    """
    pgy file to npy
    """
    with open(pgy_file) as f:
        lines = f.readlines()
    lines = lines[1:] # remove header
    np_data = np.zeros((len(lines), 3), np.float64)
    for i, line in enumerate(lines):
        l = []
        for t in line.split():
            try:
                l.append(float(t))
            except ValueError:
                pass
        np_data[i] = np.asarray(l[1:])
    return np_data

plot_using = 'plotly_light'

In [ ]:
# Some useful color arrays
colour_white = np.array([[1, 1, 1]], dtype=np.float64).swapaxes(1, 0)
colour_red = np.array([[1, 0, 0]], dtype=np.float64).swapaxes(1, 0)
colour_black = np.array([[0, 0, 0]], dtype=np.float64).swapaxes(1, 0)
colour_blue = np.array([[0, 0, 1]], dtype=np.float64).swapaxes(1, 0)
colour_green = np.array([[0, 1, 0]], dtype=np.float64).swapaxes(1, 0)

## load cleaned point cloud

In [ ]:
# load clusters
clusters_rerun_points_labelled = np.fromfile('ExtractedTransducerClustersPCs-jul24.npy').reshape((-1,4))
refined_points = clusters_rerun_points_labelled[:,:3]
labels_rerun = clusters_rerun_points_labelled[:,-1]

## test first fit

In [ ]:
# create transducer object
r = 5 #4.44
center[0] = 0
center[1] = 0

# cap
x = np.linspace(-5,5,50)
y = np.linspace(-5,5,50)
xy = np.zeros((len(x)*len(y),3))
for i in range(len(x)):
    j = i*len(y)
    xy[j:j+len(y),0] = x[i]
    xy[j:j+len(y),1] = y

cap = xy[np.sum(xy**2, axis=-1) < r**2]

# cyl (perhaps disk only is better?)
z = np.linspace(0, 5, 12)
cyl = create_cylinder(z, r)

ref = np.zeros((len(cyl)+len(cap), 3))
ref[:len(cyl)] = cyl
ref[len(cyl):] = cap

print(f"Number of points in reference cylinder {len(ref)}")

In [ ]:
np.save('cap.npy', cap)
np.save('cyl.npy', cyl)

In [ ]:
# check single pointcloud
test = refined_points[labels_rerun==26]
fig = plot_PCs([test])
fig.show()

In [ ]:
# test fitting cap
test = refined_points[labels_rerun==26] # select a decent one, I have not found a way
                                   # to QC this clustering

# remove excess points?
print(f"Number of points in observed cluster {len(test)}")

center_test = np.mean(test, axis=0)
test -= center_test
start = [0,0,0,0,0,0] # it will be important to have a rough starting point for 
                      # each! the original geomtery will be very helpful

print('start_fitting...')
print(f'center cap: {cap.mean(axis=0)}')
Cap_PC = affinePC(test, cap)
Cap_PC.fit(start=start, bounds=[(-2*np.pi,2*np.pi)]*3+[(None,None)]*3, method='naive')
print(f"Affine parametes [rot_x, rot_y, rot_z, t_x, t_y, t_z]: \n{Cap_PC.params}")
reg_pc = Cap_PC.apply_aff()
params_transd = Cap_PC.params.copy()

# construct open3D objects
reg = o3d.utility.Vector3dVector(reg_pc)
pcd_reg = o3d.geometry.PointCloud(reg)
pcd_reg = pcd_reg.paint_uniform_color(colour_blue)
test_ = o3d.utility.Vector3dVector(test)
pcd_test = o3d.geometry.PointCloud(test_)
pcd_test = pcd_test.paint_uniform_color(colour_green)

fig = plot_PCs([reg_pc, test])
fig.show()

In [ ]:
# fitting cylinder
start = params_transd # it will be important to have a rough starting point for 
                      # each! the original geomtery will be very helpful
dx = 2
dphi = 0.2
bounds_t = [(start[3]-dx, start[3]+dx), 
            (start[4]-dx, start[4]+dx),
            (start[5]-dx, start[5]+dx)]
bounds_r = [(start[0]-dphi, start[0]+dphi), 
            (start[1]-dphi, start[1]+dphi),
            (start[2]-dphi, start[2]+dphi)]

# determine polarity
ref[:len(cyl)] = cyl
ref[len(cyl):] = cap

normal_cap = np.cross(reg_pc[0], reg_pc[1]) # take any 2 cap points
normal_cap /= np.linalg.norm(normal_cap)
trans = params_transd[3:]/np.linalg.norm(params_transd[3:])

if np.dot(normal_cap, trans) < 0:
    print("flip cylinder geom...")
    ref[:,2] = -ref[:,2]

print('start_fitting...')
Cyl_PC = affinePC(test, ref)
Cyl_PC.center_ct = np.array([0,0,0])
Cyl_PC.fit(start=start, bounds=bounds_r+bounds_t, method='naive')
print(f"Affine parametes [rot_x, rot_y, rot_z, t_x, t_y, t_z]: \n{Cyl_PC.params}")
regrefined_pc = Cyl_PC.apply_aff()

# construct open3D objects
reg_ = o3d.utility.Vector3dVector(regrefined_pc)
pcd_regrefined = o3d.geometry.PointCloud(reg_)
pcd_regrefined = pcd_regrefined.paint_uniform_color(colour_blue)

fig = plot_PCs([regrefined_pc, test])
fig.show()

In [ ]:
count = 0
labels_fit = list(set(labels_rerun)) 
errors= []

# slow, use script on server!
for label in labels_fit:

    test = refined_points[labels_rerun==label] # select a decent one, I have not found a way
                                    # to QC this clustering

    try: 
        os.mkdir(f"fittedTransducers/{label}/") # create dir
    except:
        pass

    # remove excess points?
    print(f"Number of points in observed cluster {len(test)}")

    center_test = np.mean(test, axis=0)
    test -= center_test
    start = [np.pi/3,0,np.pi/3,0,0,0] # it will be important to have a rough starting point for 
                        # each! the original geomtery will be very helpful

    print('start_fitting...')
    print(f'center cap: {cap.mean(axis=0)}')
    Cap_PC = affinePC(test, cap)
    Cap_PC.fit(start=start, bounds=[(-2*np.pi,2*np.pi)]*3+[(None,None)]*3, method='naive')
    print(f"Affine parametes [rot_x, rot_y, rot_z, t_x, t_y, t_z]: \n{Cap_PC.params}")
    reg_pc = Cap_PC.apply_aff()
    params_transd = Cap_PC.params.copy()

    bucket = {"location" : center_test,
              "Objects"  : Cap_PC}
    with open(f"fittedTransducers/{label}/transCap_{label}.pk", 'wb') as handle:
        pickle.dump(bucket, handle)

    fig = plot_PCs([reg_pc, test])
    fig.write_html(f"fittedTransducers/{label}/transCap_{label}.html")

    # refit with extended ref
    start = params_transd # it will be important to have a rough starting point for 
                        # each! the original geomtery will be very helpful
    dx = 2
    dphi = 0.2
    bounds_t = [(start[3]-dx, start[3]+dx), 
                (start[4]-dx, start[4]+dx),
                (start[5]-dx, start[5]+dx)]
    bounds_r = [(start[0]-dphi, start[0]+dphi), 
                (start[1]-dphi, start[1]+dphi),
                (start[2]-dphi, start[2]+dphi)]

    # determine polarity
    ref[:len(cyl)] = cyl
    ref[len(cyl):] = cap

    normal_cap = np.cross(reg_pc[0], reg_pc[1]) # take any 2 cap points
    normal_cap /= np.linalg.norm(normal_cap)
    trans = params_transd[3:]/np.linalg.norm(params_transd[3:])

    if np.dot(normal_cap, trans) < 0:
        print("flip cylinder geom...")
        ref[:,2] = -ref[:,2]

    print('start_fitting...')
    Cyl_PC = affinePC(test, ref)
    Cyl_PC.center_ct = np.array([0,0,0])
    Cyl_PC.fit(start=start, bounds=bounds_r+bounds_t, method='naive')
    print(f"Affine parametes [rot_x, rot_y, rot_z, t_x, t_y, t_z]: \n{Cyl_PC.params}")
    regrefined_pc = Cyl_PC.apply_aff()
    loss = Cyl_PC.loss_l2(Cyl_PC.params, Cyl_PC.pc_ct, Cyl_PC.pc_us, Cyl_PC.center_ct, method='naive')
    if loss/len(Cyl_PC.pc_us) > 0.0004:
        count += 1
        errors.append(label)
        print(f"{i}:  {loss/len(Cyl_PC.pc_us)}")

    bucket = {"location" : center_test,
              "Objects"  : Cyl_PC}
    with open(f"fittedTransducers/{label}/transCyl_{label}.pk", 'wb') as handle:
        pickle.dump(bucket, handle)

    fig = plot_PCs([regrefined_pc, test])
    fig.write_html(f"fittedTransducers/{label}/transCyl_{label}.html")

print(errors)